<a href="https://colab.research.google.com/github/ann04ka/Labs/blob/main/Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Реализовать с помощью `Numpy` класс `MyMLP`, моделирующий работу полносвязной нейронной сети.

Реализуемый класс должен

1. Поддерживать создание любого числа слоев с любым числом нейронов. Тип инициализации весов не регламентируется.
2. Обеспечивать выбор следующих функции активации в рамках каждого слоя: `ReLU`, `sigmoid`, `linear`.
3. Поддерживать решение задачи классификации и регрессии (выбор соответствующего лосса, в том числе для задачи многоклассовой классификации).
4. В процессе обучения использовать самостоятельно реализованный механизм обратного распространения (вывод формул в формате markdown) для применения градиентного и стохастического градиентного спусков (с выбором размера батча)
5. Поддерживать использование `l1`, `l2` и `l1l2` регуляризаций.

Самостоятельно выбрать наборы данных (классификация и регрессия). Провести эксперименты (различные конфигурации сети: количество слоев, нейронов, функции активации, скорость обучения и тп. — минимум 5 различных конфигураций) и сравнить результаты работы (оценка качества модели + время обучения и инференса) реализованного класса `MyMLP` со следующими моделям (в одинаковых конфигурациях):

*   MLPClassifier/MLPRegressor из sklearn
*   TensorFlow
*   Keras
*   PyTorch

Результат представить в виде .ipynb блокнота, содержащего весь необходимый код и визуализации сравнения реализаций для рассмотренных конфигураций.


# MyMLP

In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing, load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

from sklearn.metrics import mean_squared_error, accuracy_score

In [2]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    s = sigmoid(x)
    return s * (1 - s)

def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return (x > 0).astype(float)

def linear(x):
    return x

def linear_derivative(x):
    return np.ones_like(x)

def softmax(x):
    exps = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exps / np.sum(exps, axis=1, keepdims=True)

In [71]:
class MyMLP:
    def __init__(self, layers, activation='relu', output_activation='linear',
                 loss='mse', learning_rate=0.01, batch_size=32, epochs=10,
                 l2=0.0):
        """
        :param layers: список чисел нейронов в каждом слое, например [8, 64, 1]
        :param activation: функция активации скрытых слоёв ('relu'/'sigmoid'/'linear')
        :param output_activation: функция активации выходного слоя ('linear'/'softmax')
        :param loss: тип лосса ('mse'/'cross_entropy')
        :param learning_rate: скорость обучения
        :param batch_size: размер батча
        :param epochs: число эпох
        :param l2: коэффициент L2-регуляризации
        """
        self.layers = layers
        self.activation_name = activation
        self.output_activation_name = output_activation
        self.loss_name = loss
        self.lr = learning_rate
        self.batch_size = batch_size
        self.epochs = epochs
        self.l2 = l2

        self.parameters = {}
        for i in range(len(layers) - 1):
            self.parameters[f'W{i}'] = np.random.randn(layers[i], layers[i+1]) * 0.01
            self.parameters[f'b{i}'] = np.zeros((1, layers[i+1]))

        self.activation = {'relu': relu, 'sigmoid': sigmoid, 'linear': linear}[activation]
        self.activation_derivative = {'relu': relu_derivative, 'sigmoid': sigmoid_derivative, 'linear': lambda x: 1}[activation]
        self.output_activation = {'linear': linear, 'softmax': softmax}[output_activation]

        if loss == 'mse':
            self.loss_fn = lambda y_true, y_pred: 0.5 * np.mean((y_true - y_pred)**2)
            self.dloss_fn = lambda y_true, y_pred: (y_pred - y_true) / y_true.shape[0]
        elif loss == 'cross_entropy':
            self.loss_fn = lambda y_true, y_pred: -np.mean(np.sum(y_true * np.log(y_pred + 1e-15), axis=1))
            self.dloss_fn = lambda y_true, y_pred: (y_pred - y_true) / y_true.shape[0]

    def forward(self, X):
        self.A = [X]
        self.Z = []

        for i in range(len(self.layers) - 1):
            W = self.parameters[f'W{i}']
            b = self.parameters[f'b{i}']
            z = np.dot(self.A[-1], W) + b
            self.Z.append(z)

            if i < len(self.layers) - 2:
                a = self.activation(z)
            else:
                a = self.output_activation(z)

            self.A.append(a)

        return self.A[-1]

    def backward(self, y_true):
        grads = {}
        m = y_true.shape[0]
        dA = self.dloss_fn(y_true, self.A[-1])

        for i in reversed(range(len(self.layers) - 1)):
            if i == len(self.layers) - 2:
                act_derivative = 1
            else:
                act_derivative = self.activation_derivative(self.Z[i])

            dZ = dA * act_derivative

            grads[f'dW{i}'] = np.dot(self.A[i].T, dZ) + (self.l2 * self.parameters[f'W{i}'] / m)
            grads[f'db{i}'] = np.sum(dZ, axis=0, keepdims=True)

            dA = np.dot(dZ, self.parameters[f'W{i}'].T)

        for i in range(len(self.layers) - 1):
            self.parameters[f'W{i}'] -= self.lr * grads[f'dW{i}']
            self.parameters[f'b{i}'] -= self.lr * grads[f'db{i}']

    def fit(self, X_train, y_train):
        for epoch in range(self.epochs):
            permutation = np.random.permutation(X_train.shape[0])
            X_shuffled = X_train[permutation]
            y_shuffled = y_train[permutation]

            for i in range(0, X_train.shape[0], self.batch_size):
                X_batch = X_shuffled[i:i+self.batch_size]
                y_batch = y_shuffled[i:i+self.batch_size]

                y_pred = self.forward(X_batch)

                self.backward(y_batch)

            if epoch % 10 == 0:
                y_pred = self.forward(X_train)
                loss = self.loss_fn(y_train, y_pred)
                print(f"Epoch {epoch}, Loss: {loss:.4f}")

    def predict(self, X):
        return self.forward(X)

# Сравнение

## Классификация

In [10]:
from sklearn.datasets import load_iris
data = load_iris()
X_clf, y_clf = data.data, data.target
X_clf = StandardScaler().fit_transform(X_clf)

ohe = OneHotEncoder(sparse_output=False)
y_clf_ohe = ohe.fit_transform(y_clf.reshape(-1, 1))

X_train_clf, X_test_clf, y_train_clf, y_test_clf = train_test_split(X_clf, y_clf_ohe, test_size=0.2, random_state=42)

## Регрессия

In [11]:
data = fetch_california_housing()
X_reg, y_reg = data.data, data.target
X_reg = StandardScaler().fit_transform(X_reg)
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)

## Эксперименты с разными конфигурациями

Примеры сетей:

| Название | Архитектура | Активация | Задача |
|---------|-------------|------------|--------|
| A       | [64, 32, 1] | ReLU → Linear | Regressor |
| B       | [128, 64, 3] | ReLU → Softmax | Classifier |
| C       | [64, 64, 3] | ReLU → Softmax | Classifier |
| D       | [128, 1]     | Linear       | Regressor |
| E       | [64, 32, 3] | Sigmoid → Softmax | Classifier |

In [56]:
def test_regression_model(model_class, model_params, X_train, y_train, X_test, y_test):
    model = model_class(**model_params)
    start_time = time.time()
    model.fit(X_train, y_train)
    end_time = time.time()
    preds = model.predict(X_test).flatten()
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    return rmse, end_time - start_time


def test_classification_model(model_class, model_params, X_train, y_train, X_test, y_test):
    model = model_class(**model_params)
    start_time = time.time()
    model.fit(X_train, y_train)
    end_time = time.time()
    preds = model.predict(X_test)
    if len(preds.shape) == 2 and preds.shape[1] > 1:
        preds = np.argmax(preds, axis=1)
    y_true = np.argmax(y_test, axis=1)
    acc = accuracy_score(y_true, preds)
    return acc, end_time - start_time

In [78]:
layer_configurations = {
    'regression': {
        'MyMLP_1': [X_train_reg.shape[1], 64, 1],
        'MyMLP_2': [X_train_reg.shape[1], 128, 1],
        'MyMLP_3': [X_train_reg.shape[1], 64, 32, 1],
        'MyMLP_4': [X_train_reg.shape[1], 128, 64, 1],
        'MyMLP_5': [X_train_reg.shape[1], 64, 64, 1],
        'sklearn': [64]
    },
    'classification': {
        'MyMLP_1': [X_train_clf.shape[1], 64, y_train_clf.shape[1]],
        'MyMLP_2': [X_train_clf.shape[1], 128, y_train_clf.shape[1]],
        'MyMLP_3': [X_train_clf.shape[1], 64, 32, y_train_clf.shape[1]],
        'MyMLP_4': [X_train_clf.shape[1], 128, 64, y_train_clf.shape[1]],
        'MyMLP_5': [X_train_clf.shape[1], 64, 64, y_train_clf.shape[1]],
        'sklearn': [64]
    }
}

In [82]:
print("=== РЕГРЕССИЯ: СРАВНЕНИЕ АРХИТЕКТУР ===")
results_regression = []

for name, layers in [
    ('MyMLP_1', layer_configurations['regression']['MyMLP_1']),
    ('MyMLP_2', layer_configurations['regression']['MyMLP_2']),
    ('MyMLP_3', layer_configurations['regression']['MyMLP_3']),
    ('MyMLP_4', layer_configurations['regression']['MyMLP_4']),
    ('MyMLP_5', layer_configurations['regression']['MyMLP_5'])
]:
    model = MyMLP(
        layers=layers,
        activation='relu',
        output_activation='linear',
        loss='mse',
        learning_rate=0.001,
        batch_size=32,
        epochs=20
    )

    rmse, duration = test_regression_model(MyMLP, dict(
        layers=layers,
        activation='relu',
        output_activation='linear',
        loss='mse',
        learning_rate=0.001,
        batch_size=1,
        epochs=20
    ), X_train_reg, y_train_reg, X_test_reg, y_test_reg)
    results_regression.append({'Model': name, 'RMSE': rmse, 'Time': duration, 'Framework': 'MyMLP'})

=== РЕГРЕССИЯ: СРАВНЕНИЕ АРХИТЕКТУР ===
Epoch 0, Loss: 1.0272
Epoch 10, Loss: 1.0755
Epoch 0, Loss: 1.0612
Epoch 10, Loss: 1.1402
Epoch 0, Loss: 0.6691
Epoch 10, Loss: 1.1865
Epoch 0, Loss: 0.7254
Epoch 10, Loss: 1.1769
Epoch 0, Loss: 0.7581
Epoch 10, Loss: 1.1987


In [83]:
import  pandas as pd

regression = pd.DataFrame.from_dict(results_regression)

regression

,Model,RMSE,Time,Framework
0,MyMLP_1,0.612667,30.962547,MyMLP
1,MyMLP_2,0.616186,28.319477,MyMLP
2,MyMLP_3,0.587940,44.332793,MyMLP
3,MyMLP_4,0.585242,53.900210,MyMLP
4,MyMLP_5,0.584590,47.743464,MyMLP


In [74]:
print("\n=== КЛАССИФИКАЦИЯ: СРАВНЕНИЕ АРХИТЕКТУР ===")
results_classification = []

for name, layers in [
    ('MyMLP_1', layer_configurations['classification']['MyMLP_1']),
    ('MyMLP_2', layer_configurations['classification']['MyMLP_2']),
    ('MyMLP_3', layer_configurations['classification']['MyMLP_3']),
    ('MyMLP_4', layer_configurations['classification']['MyMLP_4']),
    ('MyMLP_5', layer_configurations['classification']['MyMLP_5'])
]:
    rmse, duration = test_classification_model(MyMLP, dict(
        layers=layers,
        activation='relu',
        output_activation='softmax',
        loss='cross_entropy',
        learning_rate=0.01,
        batch_size=32,
        epochs=20
    ), X_train_clf, y_train_clf, X_test_clf, y_test_clf)
    results_classification.append({'Model': name, 'Accuracy': rmse, 'Time': duration, 'Framework': 'MyMLP'})



=== КЛАССИФИКАЦИЯ: СРАВНЕНИЕ АРХИТЕКТУР ===
Epoch 0, Loss: 1.0982
Epoch 10, Loss: 1.0966
Epoch 0, Loss: 1.0980
Epoch 10, Loss: 1.0953
Epoch 0, Loss: 1.0986
Epoch 10, Loss: 1.0985
Epoch 0, Loss: 1.0986
Epoch 10, Loss: 1.0985
Epoch 0, Loss: 1.0986
Epoch 10, Loss: 1.0985


In [75]:
classification = pd.DataFrame.from_dict(results_classification)

classification

,Model,Accuracy,Time,Framework
0,MyMLP_1,0.633333,0.011407,MyMLP
1,MyMLP_2,0.633333,0.013157,MyMLP
2,MyMLP_3,0.300000,0.016910,MyMLP
3,MyMLP_4,0.300000,0.038253,MyMLP
4,MyMLP_5,0.300000,0.027849,MyMLP


Реализация `MyMLP` позволяет запускать разные архитектуры, но требует доработки для повышения качества


Архитектура `[in, 64, 32, out]` показала себя лучше всего в задаче регрессии


## Сравнение с библиотечными


In [69]:
import time

print("=== РЕГРЕССИЯ ===")

# MyMLP
model_mlp_np = MyMLP(
    layers=layer_configurations['regression']['MyMLP_3'],
    activation='relu',
    output_activation='linear',
    loss='mse',
    learning_rate=0.01,
    batch_size=1,
    epochs=20
)
start_time = time.time()
model_mlp_np.fit(X_train_reg, y_train_reg)
end_time = time.time()
preds_np = model_mlp_np.predict(X_test_reg).flatten()
score_np = mean_squared_error(y_test_reg, preds_np)
print(f"MyMLP (NumPy): RMSE={np.sqrt(score_np):.4f}, Time={end_time - start_time:.2f}s")

# sklearn
from sklearn.neural_network import MLPRegressor
model_sklearn = MLPRegressor(hidden_layer_sizes=layer_configurations['regression']['sklearn'], max_iter=50, random_state=42)
start_time = time.time()
model_sklearn.fit(X_train_reg, y_train_reg)
end_time = time.time()
preds_sklearn = model_sklearn.predict(X_test_reg)
score_sklearn = mean_squared_error(y_test_reg, preds_sklearn)
print(f"MLPRegressor (sklearn): RMSE={np.sqrt(score_sklearn):.4f}, Time={end_time - start_time:.2f}s")

=== РЕГРЕССИЯ ===
MyMLP (NumPy): RMSE=4.9181, Time=48.35s
MLPRegressor (sklearn): RMSE=0.5766, Time=2.08s


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


In [70]:
print("=== КЛАССИФИКАЦИЯ ===")

# MyMLP
model_mlp_np = MyMLP(
    layers=layer_configurations['classification']['MyMLP_2'],
    activation='relu',
    output_activation='softmax',
    loss='cross_entropy',
    learning_rate=0.01,
    batch_size=32,
    epochs=50
)

start_time = time.time()
model_mlp_np.fit(X_train_clf, y_train_clf)
end_time = time.time()
preds_np = model_mlp_np.predict(X_test_clf)
y_pred_np = np.argmax(preds_np, axis=1)
y_true = np.argmax(y_test_clf, axis=1)
acc_np = accuracy_score(y_true, y_pred_np)
print(f"MyMLP: Accuracy={acc_np:.4f}, Time={end_time - start_time:.2f}s")

# sklearn
from sklearn.neural_network import MLPClassifier
model_sklearn = MLPClassifier(hidden_layer_sizes=layer_configurations['classification']['sklearn'],
                              max_iter=50, random_state=42)
start_time = time.time()
model_sklearn.fit(X_train_clf, y_train_clf)
end_time = time.time()

preds_proba = model_sklearn.predict_proba(X_test_clf)
preds_sklearn = np.argmax(preds_proba, axis=1)
acc_sklearn = accuracy_score(y_true, preds_sklearn)
print(f"MLPClassifier (sklearn): Accuracy={acc_sklearn:.4f}, Time={end_time - start_time:.2f}s")

=== КЛАССИФИКАЦИЯ ===
MyMLP: Accuracy=0.3333, Time=0.05s
MLPClassifier (sklearn): Accuracy=0.9333, Time=0.05s


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


In [54]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# TensorFlow/Keras
model_keras = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_clf.shape[1],)),
    Dense(y_train_clf.shape[1], activation='softmax')
])
model_keras.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
start_time = time.time()
model_keras.fit(X_train_clf, y_train_clf, epochs=20, batch_size=1, verbose=0)
end_time = time.time()
_, acc_keras = model_keras.evaluate(X_test_clf, y_test_clf, verbose=0)
print(f"Keras (TensorFlow): Accuracy={acc_keras:.4f}, Time={end_time - start_time:.2f}s")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Keras (TensorFlow): Accuracy=1.0000, Time=7.01s


In [55]:
import torch
import torch.nn as nn
import torch.optim as optim

class SimpleNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        return self.net(x)

X_train_tensor = torch.tensor(X_train_clf, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_clf, dtype=torch.float32)
train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=True)

net = SimpleNet(X_train_clf.shape[1], 64, y_train_clf.shape[1])
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.01)

start_time = time.time()
for epoch in range(20):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, torch.argmax(labels, dim=1))
        loss.backward()
        optimizer.step()
end_time = time.time()

with torch.no_grad():
    test_input = torch.tensor(X_test_clf, dtype=torch.float32)
    pred = net(test_input).argmax(dim=1)
    true = torch.tensor(y_true)
    acc_pt = (pred == true).float().mean().item()
print(f"PyTorch: Accuracy={acc_pt:.4f}, Time={end_time - start_time:.2f}s")

PyTorch: Accuracy=0.9667, Time=4.32s


## Выводы


| Модель | Задача | Accuracy / RMSE | Время обучения |
|-------|--------|------------------|----------------|
| **MyMLP (NumPy)** | Регрессия | RMSE = 0.8908 | 37.54 с |
| **MLPRegressor (sklearn)** | Регрессия | RMSE = 0.5766 | 3.06 с |
| **MyMLP (NumPy)** | Классификация | Accuracy = 0.3667 | 0.05 с |
| **MLPClassifier (sklearn)** | Классификация | Accuracy = 0.9333 | 0.05 с |
| **Keras (TensorFlow)** | Классификация | Accuracy = 1.0000 | 7.01 с |
| **PyTorch** | Классификация | Accuracy = 0.9667 | 4.32 с |

---


# Регрессия (California Housing)

### MyMLP:
- RMSE: **0.8908**
- Время: **37.54 секунды**

### sklearn:
- RMSE: **0.5766**
- Время: **3.06 секунды**

- `MyMLP` уступает по качеству и скорости `MLPRegressor` из `sklearn`

---

# Классификация (Iris)

### MyMLP:
- Accuracy: **0.3667** (почти случайное угадывание для 3 классов!)
- Время: **0.05 с**

### sklearn:
- Accuracy: **0.9333**
- Время: **0.05 с**

### TensorFlow/Keras:
- Accuracy: **1.0000**
- Время: **7.01 с**

### PyTorch:
- Accuracy: **0.9667**
- Время: **4.32 с**


`MyMLP` обучается хуже чем внешние фреймворки

---

# Выводы

В ходе выполнения лабораторной работы была разработана и протестирована полносвязная нейронная сеть `MyMLP`, реализованная с использованием только библиотеки `NumPy`. Модель была обучена на задачах регрессии (`California Housing`) и многоклассовой классификации (`Iris`).  


Результаты показали, что:

- Реализация `MyMLP` позволяет запустить обучение, но требует доработки для повышения качества.
- Фреймворки `scikit-learn`, `TensorFlow`, `Keras`, `PyTorch` показывают значительно более высокую скорость и качество.

Кастомный класс для полносвязной нейронной сети можно улучшить, если:

- Улучшить инициализацию весов
- Расширить набор поддерживаемых функций активации и лоссов
- Добавить поддержку современных оптимизаторов (Adam, Momentum)

---